draft augmented version of notebook 9b using Simple Pipeline Executor (SPE)

In [1]:
# copied from existing notebooks 9a, 9b
! echo $IMAGE_DESCRIPTION

w_2022_40


In [2]:
# copied from existing notebook 9a
#import getpass
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas
from astropy.time import Time

import lsst.geom
import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler, DatasetType, CollectionType
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline, Instrument

In [3]:
my_collection_identifier = 'custom_coadd_window1_test1'
my_username = os.environ['USER']
my_collection = 'u/' + my_username + '/' + my_collection_identifier

In [4]:
print(my_collection)

u/ameisner/custom_coadd_window1_test1


In [5]:
ordered_collections_list = [my_collection, '2.2i/runs/DP0.2']
### ordered_collections_list = ['2.2i/runs/DP0.2'] # this was just a test, don't actually want to do this !!

In [6]:
butler = Butler('dp02', collections=ordered_collections_list)

In [7]:
my_tract = 4431
my_patch = 17
my_visits_tupleString = '(919515,924057,924085,924086,929477,930353)' # not used until later
my_dataId = {'band': 'i', 'tract': my_tract, 'patch': my_patch}
my_deepCoadd = butler.get('deepCoadd', my_dataId)

In [8]:
my_deepCoadd_inputs = my_deepCoadd.getInfo().getCoaddInputs()
my_deepCoadd_inputs.visits.asAstropy()

id,bbox_min_x,bbox_min_y,bbox_max_x,bbox_max_y,goodpix,weight,filter
,pix,pix,pix,pix,,,
int64,int32,int32,int32,int32,int32,float64,str32
919515,11900,7900,16099,12099,8982709,3.4656688819793495,i_sim_1.4
924057,11900,7900,16099,12099,16098179,4.384267091685517,i_sim_1.4
924085,11900,7900,16099,12099,831332,4.446833161599578,i_sim_1.4
924086,11900,7900,16099,12099,16136708,4.550420295334223,i_sim_1.4
929477,11900,7900,16099,12099,16280498,4.051326013718346,i_sim_1.4
930353,11900,7900,16099,12099,16076133,3.7685753871220466,i_sim_1.4


In [9]:
len(my_deepCoadd_inputs.visits) # should be 6 or something has gone wrong...

6

In [10]:
del butler

In [ ]:
# relevant subset of $DRP_PIPE_DIR/ingredients/LSSTCam-imSim/DRP.yaml

#  step3:
#    subset:
#      - makeWarp          ### already done in 9a
#      - assembleCoadd     ### already done in 9a
#      - detection
#      - mergeDetections
#      - deblend
#      - measure
#      - mergeMeasurements
#      - forcedPhotCoadd
#      - transformObjectTable
#      - writeObjectTable
#      - consolidateObjectTable
#      - healSparsePropertyMaps    ### not necessary based on Jira ticket description
#      - selectGoodSeeingVisits    ### not necessary based on Jira ticket description
#      - templateGen               ### not necessary based on Jira ticket description
#    description: |

In [11]:
yaml_file = '$DRP_PIPE_DIR/ingredients/LSSTCam-imSim/DRP.yaml'
steps = 'detection' # try this as an initial test case
# steps = 'detection,mergeDetections,deblend,measure,mergeMeasurements,forcedPhotCoadd,transformObjectTable,writeObjectTable,consolidateObjectTable'
my_uri = yaml_file + '#' + steps
print(my_uri)

$DRP_PIPE_DIR/ingredients/LSSTCam-imSim/DRP.yaml#detection


In [12]:
myDetectionPipeline = Pipeline.from_uri(my_uri)

In [ ]:
#configs from 9b: 
#  (1) characterizeimage -- 1 psfiteration    ### i think this is actually irrelevant when using the SPE framework?
#  (2) detection -- thresholdValue = 10 (10 sigma)
#  (3) detection -- thresholdType = "stdev"

In [ ]:
# myDetectionPipeline.addConfigOverride('CharacterizeImage', 'psfIterations', 1) # this crashes with "LookupError: There are no tasks labeled 'CharacterizeImage' in the pipeline"
#myDetectionPipeline.addConfigOverride('characterizeImage', 'psfIterations', 1) # this crashes with "LookupError: There are no tasks labeled 'characterizeImage' in the pipeline"

In [13]:
myDetectionPipeline.addConfigOverride('detection', 'detection.thresholdValue', 10)
myDetectionPipeline.addConfigOverride('detection', 'detection.thresholdType', "stdev")
?myDetectionPipeline.addConfigOverride

Signature:
myDetectionPipeline.addConfigOverride(
    label: 'str',
    key: 'str',
    value: 'object',
) -> 'None'
Docstring:
Apply single config override.

Parameters
----------
label : `str`
    Label of the task.
key: `str`
    Fully-qualified field name.
value : object
    Value to be given to a field.
File:      /opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/pipe_base/gbbd6c36fd7+a713711289/python/lsst/pipe/base/pipeline.py
Type:      method


In [14]:
queryString = f"tract = {my_tract} AND patch = {my_patch} AND " + \
              f"visit in {my_visits_tupleString} AND skymap = 'DC2'"

print(queryString)

tract = 4431 AND patch = 17 AND visit in (919515,924057,924085,924086,929477,930353) AND skymap = 'DC2'


In [15]:
my_outputCollection = 'u/ameisner/nb9a_spe'

In [16]:
simpleButler = SimplePipelineExecutor.prep_butler('dp02', 
                                                  inputs=ordered_collections_list, 
                                                  output=my_outputCollection)

In [17]:
simpleButler.registry.getCollectionChain(my_outputCollection)

CollectionSearch(('u/ameisner/nb9a_spe/20230508T074006Z', 'u/ameisner/custom_coadd_window1_test1', '2.2i/runs/DP0.2'))

In [79]:
for c in sorted(simpleButler.registry.queryCollections(my_outputCollection+'*')):
    print('Found: ', c)

Found:  u/ameisner/nb9a_spe
Found:  u/ameisner/nb9a_spe/20230508T070540Z
Found:  u/ameisner/nb9a_spe/20230508T071351Z
Found:  u/ameisner/nb9a_spe/20230508T072348Z
Found:  u/ameisner/nb9a_spe/20230508T073025Z
Found:  u/ameisner/nb9a_spe/20230508T074006Z


In [19]:
%%time
spe = SimplePipelineExecutor.from_pipeline(myDetectionPipeline, 
                                           where=queryString, 
                                           butler=simpleButler)

/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/pipe_tasks/g369a80f31c+b9be13ffc2/python/lsst/pipe/tasks/multiBand.py:261: UserWarning: MeasureMergedCoaddSourcesConnections.defaultTemplates is deprecated and no longer used. Use MeasureMergedCoaddSourcesConfig.inputCatalog.
  warnings.warn("MeasureMergedCoaddSourcesConnections.defaultTemplates is deprecated and no longer used. "


CPU times: user 647 ms, sys: 49.9 ms, total: 697 ms
Wall time: 1.11 s


In [20]:
print(spe.quantum_graph.outputQuanta)
len(spe.quantum_graph.outputQuanta)

[QuantumNode(quantum=Quantum(taskName=lsst.pipe.tasks.multiBand.DetectCoaddSourcesTask, dataId={band: 'i', skymap: 'DC2', tract: 4431, patch: 17}), taskDef=TaskDef(lsst.pipe.tasks.multiBand.DetectCoaddSourcesTask, label=detection), nodeId=441d8a83-fbac-41f0-8dee-8d61f4a6e6d4)]


1

In [21]:
%%time
quanta = spe.run()

CPU times: user 52.4 s, sys: 3.36 s, total: 55.8 s
Wall time: 58.8 s


In [22]:
for q, quantum in enumerate(quanta):
    print(q, quantum.taskName)

0 lsst.pipe.tasks.multiBand.DetectCoaddSourcesTask


In [24]:
index = 0
for output in quanta[0].outputs:
    print(output)

DatasetType('detection_log', {band, skymap, tract, patch}, ButlerLogRecords)
DatasetType('deepCoadd_det', {band, skymap, tract, patch}, SourceCatalog)
DatasetType('detection_metadata', {band, skymap, tract, patch}, PropertySet)
DatasetType('deepCoadd_calexp', {band, skymap, tract, patch}, ExposureF)
DatasetType('deepCoadd_calexp_background', {band, skymap, tract, patch}, Background)


In [69]:
#del butler
butler = Butler('dp02', collections=['u/ameisner/nb9a_spe'])

In [71]:
my_new_deepCoadd_calexp = simpleButler.get(quanta[index].outputs['deepCoadd_calexp'][0])
